# Gateway Client Example
Here is a notebook demonstrating the fundamentals of the `GatewayClient` object.

This class is a convenience wrapper around the various `HTTP` requests used to interact with a running `GatewayServer` (with `REST` endpoints enabled). It relies on the [httpx](https://www.python-httpx.org/) and [aiohttp](https://docs.aiohttp.org/en/stable/index.html) libraries, and derives most of its functionality from the [openapi](https://www.openapis.org/) specification provided on the running `GatewayServer` (usually available at `/openapi.json`).


## Client Configuration
A `GatewayClient` is configured via a `GatewayClientConfig`, a minimal [pydantic](https://docs.pydantic.dev/) model to specify details such as protocol (`http`/`https`), host, port, etc. By default, we should provide `host` and `port`. 

We can also specify in the config `return_raw_json`, which specifies whether we would like to return the raw json response, or a `ResponseWrapper` object for `REST` requests. The `ResponseWrapper` object can provide both the raw json message, as well as a pandas dataframe. The `ResponseWrapper` contains additional type information which will create column names and utilize the correct data type for the constructed pandas dataframe.


## Client Methods

A client as a small number of general-purpose methods. In alphabetical order:

- `controls`: managment controls for monitoring/configurating the running server
- `last`: get the last ticked data value on a channel
- `lookup`: lookup a piece of data by `id`
- `next`: get the next ticked data value on a channel
- `send`: send some data onto a channel
- `state`: get the value of a given channel's state accumulator


Additionally, a client has some streaming methods available when websockets are configured:

- `stream`: call a callback when a channel ticks
- `subscribe`: subscribe to data on a channel
- `unsubscribe`: unsubscribe to data on a channel

Let's explore some of the functionality of the basic demo server. To start, we should run the demo server in another process:

```bash
python -m csp_gateway.server.demo
```

By default, this will run the server on `localhost:8000`.

## Imports
All the important objects are hoisted to the top level `csp_gateway`. Lets import and setup our client.

In [1]:
from csp_gateway import GatewayClient, GatewayClientConfig

In [2]:
config = GatewayClientConfig(host="localhost", port=8000, return_raw_json=False)
client = GatewayClient(config)

The first time we use our client, it will take a little longer than usual as it tries to interrogate the running server's `openapi` specification for available methods. Once done, our request will go through, and subsequent requests will leverage this specification. Let's start with some simple status checks. If we're curious about available endpoints, we can navigate to [http://localhost:8000/redoc](http://localhost:8000/redoc) (or generally `http://<hostname>:<port>/redoc` if we're running on a different host)

In [3]:
# heartbeat check
client.controls("heartbeat").as_json()

[{'id': '2319375449026723841',
  'timestamp': '2024-02-08T15:42:24.529000+00:00',
  'name': 'heartbeat',
  'status': 'ok',
  'data': {}}]

In [4]:
# openapi spec
from IPython.display import JSON
JSON(client._openapi_spec)

<IPython.core.display.JSON object>

In [5]:
# machine stats
client.controls("stats").as_json()

[{'id': '2319375449026723842',
  'timestamp': '2024-02-08T15:42:24.707000+00:00',
  'name': 'stats',
  'status': 'ok',
  'data': {'cpu': 9.5,
   'memory': 70.0,
   'memory-total': 29.98,
   'now': '2024-02-08T15:42:24.709000+00:00',
   'csp-now': '2024-02-08T15:42:24.707178+00:00',
   'host': 'myhost',
   'user': 'csp-gateway-user'}}]

## Last, State, Lookup, Send
Let's look at what channels we have available for `last`:

In [6]:
client.last().as_json()

['never_ticks', 'example', 'example_list', 'str_basket', 'controls', 'basket']

In [7]:
client.last("example").as_json()

[{'id': '2319375449093843665',
  'timestamp': '2024-02-08T15:42:24.674000+00:00',
  'x': 2740,
  'y': '274027402740',
  'internal_csp_struct': {'z': 12},
  'data': [0.04214403621123841,
   0.2777565518959145,
   0.8924844325909429,
   0.34476509440152614,
   0.20822638755894596,
   0.9031877679300264,
   0.6124216455541363,
   0.9848707643841728,
   0.618990569185841,
   0.5582898776824039],
  'mapping': {'2740': 2740}}]

In [8]:
client.last("basket").as_json()

[{'id': '2319375449093843662',
  'timestamp': '2024-02-08T15:42:24.674000+00:00',
  'x': 2740,
  'y': '2740',
  'internal_csp_struct': {'z': 12},
  'data': [0.4358136168874479,
   0.9866855468623034,
   0.7370733232695977,
   0.2473537128693415,
   0.33386372679049414,
   0.855059230303771,
   0.23094109313426026,
   0.15447614788689634,
   0.028364551604262656,
   0.3461902446665106],
  'mapping': {'2740': 2740}},
 {'id': '2319375449093843663',
  'timestamp': '2024-02-08T15:42:24.674000+00:00',
  'x': 2740,
  'y': '27402740',
  'internal_csp_struct': {'z': 12},
  'data': [0.6377430109697712,
   0.9945325429350265,
   0.21250129814975605,
   0.7428201790683369,
   0.748495275897516,
   0.7495896160497952,
   0.052122337434189814,
   0.28571381888827774,
   0.15916263368074623,
   0.5859045937429175],
  'mapping': {'2740': 2740}},
 {'id': '2319375449093843664',
  'timestamp': '2024-02-08T15:42:24.674000+00:00',
  'x': 2740,
  'y': '274027402740',
  'internal_csp_struct': {'z': 12},
  'd

In [9]:
client.last("basket").as_pandas_df()

,id,timestamp,x,y,data,internal_csp_struct.z,mapping.2740,internal_csp_struct,mapping
0,2319375449093843662,2024-02-08T15:42:24.674000+00:00,2740,2740,"[0.4358136168874479, 0.9866855468623034, 0.737...",12,2740,NaN,NaN
1,2319375449093843663,2024-02-08T15:42:24.674000+00:00,2740,27402740,"[0.6377430109697712, 0.9945325429350265, 0.212...",12,2740,NaN,NaN
2,2319375449093843664,2024-02-08T15:42:24.674000+00:00,2740,274027402740,"[0.09866777488177447, 0.9338912139505706, 0.40...",12,2740,NaN,NaN


State channels are used to perform state accumulation over a number of ticks. The example server doesn't do anything too interesting, but we can access these channels nevertheless.

In [10]:
client.state().as_json()

['example']

In [11]:
client.state("example").as_pandas_df().tail()

# We note that there are a large number of columns in the above dataframe.
# This is because `mapping` is a dict with different keys for eery row.
# To accomodate all of them, the returned pandas dataframe has a column for any key present in the `mapping` attribute of any `ExampleData` Struct

,id,timestamp,x,y,data,internal_csp_struct.z,mapping.1,mapping.2,mapping.3,mapping.4,...,mapping.2733,mapping.2734,mapping.2735,mapping.2736,mapping.2737,mapping.2738,mapping.2739,mapping.2740,internal_csp_struct,mapping
2737,2319375449093843649,2024-02-08T15:42:20.674000+00:00,2736,273627362736,"[0.49893600652720127, 0.853167719853599, 0.315...",12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2736.0,NaN,NaN,NaN,NaN,NaN,NaN
2738,2319375449093843653,2024-02-08T15:42:21.674000+00:00,2737,273727372737,"[0.5454735801588998, 0.9131743196563596, 0.793...",12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2737.0,NaN,NaN,NaN,NaN,NaN
2739,2319375449093843657,2024-02-08T15:42:22.674000+00:00,2738,273827382738,"[0.9647636482625623, 0.5143565076074796, 0.187...",12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2738.0,NaN,NaN,NaN,NaN
2740,2319375449093843661,2024-02-08T15:42:23.674000+00:00,2739,273927392739,"[0.11711046455122953, 0.4766317339540975, 0.78...",12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2739.0,NaN,NaN,NaN
2741,2319375449093843665,2024-02-08T15:42:24.674000+00:00,2740,274027402740,"[0.04214403621123841, 0.2777565518959145, 0.89...",12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2740.0,NaN,NaN


We can lookup individual data points using `lookup`. This is a bit of a silly example when we're just looking at a single channel, but can be valuable when you have lots of interconnected channels.

In [12]:
# get the last tick, then lookup by id
last = client.last("example").as_json()[0]
last_id = last["id"]
client.lookup("example", last_id).as_json()

[{'id': '2319375449093843677',
  'timestamp': '2024-02-08T15:42:27.674000+00:00',
  'x': 2743,
  'y': '274327432743',
  'internal_csp_struct': {'z': 12},
  'data': [0.11397482092369415,
   0.8082756046612577,
   0.5269320054610495,
   0.09017303257799603,
   0.8346823428352325,
   0.12803545825097729,
   0.8563661560959381,
   0.8337489771318026,
   0.9665893466463059,
   0.7835381554236741],
  'mapping': {'2743': 2743}}]

Finally, we can send our own data into the API using `send`.

In [13]:
client.send(
    "example",
    {
        "x": 12,
        "y": "HEY!",
        "internal_csp_struct": {"z": 13}
    }
)

client.state("example").as_pandas_df().tail()

,id,timestamp,x,y,data,internal_csp_struct.z,mapping.1,mapping.2,mapping.3,mapping.4,...,mapping.2737,mapping.2738,mapping.2739,mapping.2740,mapping.2741,mapping.2742,mapping.2743,mapping.2744,internal_csp_struct,mapping
2742,2319375449093843669,2024-02-08T15:42:25.674000+00:00,2741,274127412741,"[0.7755552349359216, 0.1125930704396867, 0.339...",12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2741.0,NaN,NaN,NaN,NaN,NaN
2743,2319375449093843673,2024-02-08T15:42:26.674000+00:00,2742,274227422742,"[0.6067900468053619, 0.7816859434221093, 0.536...",12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2742.0,NaN,NaN,NaN,NaN
2744,2319375449093843677,2024-02-08T15:42:27.674000+00:00,2743,274327432743,"[0.11397482092369415, 0.8082756046612577, 0.52...",12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2743.0,NaN,NaN,NaN
2745,2319375449093843681,2024-02-08T15:42:28.674000+00:00,2744,274427442744,"[0.4131649893399658, 0.3784834347835063, 0.987...",12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2744.0,NaN,NaN
2746,2319375449093843682,2024-02-08T15:42:28.846000+00:00,12,HEY!,[],13,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The REST API uses pydantic validation for `send` requests. Since `ExampleData` has a `__validators__` attribute defined, the pydantic version of the GatewayStruct has those functions ran for validation before propagating the sent value through the graph. `ExampleData` has validation performed that asserts the value of `x` is not negative. When we try to pass a negative value in, we get an error on the send, but the graph does not crash. The details of the error are provided in the response.

In [14]:
client.send("example", {"x": -12, "y": "HEY!"})

ServerUnprocessableException: [{'loc': ['body'], 'msg': 'value is not a valid list', 'type': 'type_error.list'}, {'loc': ['body', 'x'], 'msg': 'value must be non-negative.', 'type': 'value_error'}]

## Next
The running `GatewayServer` is a synchronous system, and we're interacting it via asynchronous `REST` requests. However, we can still perform actions like "wait for the next tick". This can be dangerous and lead to race conditions, but it can still be useful in certain circumstances.

In [15]:
client.next("example").as_json()

[{'id': '2319375449093843978',
  'timestamp': '2024-02-08T15:43:42.674000+00:00',
  'x': 2818,
  'y': '281828182818',
  'internal_csp_struct': {'z': 12},
  'data': [0.04177423824388882,
   0.9576947646141436,
   0.8797403395027252,
   0.07591623282958704,
   0.9012930744265685,
   0.18036455365706483,
   0.8368363380941581,
   0.2958674194835621,
   0.7139586435389245,
   0.7923286062539309],
  'mapping': {'2818': 2818}}]

Note that this call will **block** until the next value ticks.

## Streaming
If our webserver is configured with websockets, we can also stream data out of channels. A simple example that prints out channel data is provided.

In [16]:
client.stream(channels=["example"], callback=print)

{'channel': 'example', 'data': [{'id': '2319375449093843998', 'timestamp': '2024-02-08T15:43:47.674000+00:00', 'x': 2823, 'y': '282328232823', 'internal_csp_struct': {'z': 12}, 'data': [0.7733188729889257, 0.3505657636995222, 0.37947167012560834, 0.5813803503480363, 0.4224356797080008, 0.7882237596018704, 0.7501837172662043, 0.3014755082030406, 0.11662082552665554, 0.1760084143205467], 'mapping': {'2823': 2823}}]}
{'channel': 'example', 'data': [{'id': '2319375449093844002', 'timestamp': '2024-02-08T15:43:48.674000+00:00', 'x': 2824, 'y': '282428242824', 'internal_csp_struct': {'z': 12}, 'data': [0.7935760696606141, 0.04502649843404605, 0.5772625402239133, 0.6083217755224994, 0.949351551805679, 0.2619775463100148, 0.6036207137382622, 0.0005275136962938909, 0.827541831606724, 0.88364890012582], 'mapping': {'2824': 2824}}]}
{'channel': 'example', 'data': [{'id': '2319375449093844006', 'timestamp': '2024-02-08T15:43:49.674000+00:00', 'x': 2825, 'y': '282528252825', 'internal_csp_struct': 

## Asynchronous client
All of the above can also be used in an `async` fashion. Note that by default, the `GatewayClient` class is an alias for the `SyncGatewayClient` class. The only differences are:

- all methods are `async` instead of synchronous
- `stream` is an infinite generator, rather than callback-based (so takes no `callback` argument)

In [17]:
from csp_gateway import AsyncGatewayClient

In [18]:
async_client= AsyncGatewayClient(config)

In [19]:
async def print_all():
    async for datum in async_client.stream(channels=["example", "example_list"]):
        print(datum)

In [20]:
await print_all()

{'channel': 'example_list', 'data': [{'id': '2319375449093844034', 'timestamp': '2024-02-08T15:43:56.674000+00:00', 'x': 2832, 'y': '283228322832', 'internal_csp_struct': {'z': 12}, 'data': [0.6036805890478953, 0.6749444877468045, 0.5497958103280356, 0.7245526415750495, 0.8203822683954279, 0.7692240209863609, 0.6725744504378558, 0.7092152352091319, 0.22125780238809134, 0.8010351708291975], 'mapping': {'2832': 2832}}]}
{'channel': 'example', 'data': [{'id': '2319375449093844034', 'timestamp': '2024-02-08T15:43:56.674000+00:00', 'x': 2832, 'y': '283228322832', 'internal_csp_struct': {'z': 12}, 'data': [0.6036805890478953, 0.6749444877468045, 0.5497958103280356, 0.7245526415750495, 0.8203822683954279, 0.7692240209863609, 0.6725744504378558, 0.7092152352091319, 0.22125780238809134, 0.8010351708291975], 'mapping': {'2832': 2832}}]}
{'channel': 'example_list', 'data': [{'id': '2319375449093844038', 'timestamp': '2024-02-08T15:43:57.674000+00:00', 'x': 2833, 'y': '283328332833', 'internal_csp